# RSS: Economista

## 1. Import Libraries

In [18]:
import feedparser
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import datetime
from termcolor import colored
from IPython.display import display, Markdown, Latex, HTML
import re
from IPython.display import Image
import urllib
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import numpy as np
from PIL import Image

In [19]:
def get_only_text1(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    return [text,soup.title.text]

def get_img(url):
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    if soup.find('div',class_='articleHero') != None:
        if soup.find('div',class_='articleHero').find('div',class_='articleImage') != None:
            return soup.find('div',class_='articleHero').find('div',class_='articleImage').find('img')['data-src']
        else: None
    else:
        return None

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def string_found(string1, string2):
    
    if re.search(r"\b" + re.escape(string1) + r"\b", string2.lower()):
        return True
    return False     
    
def bank_tags(text):
    tags = []
    dict_banks = {'Santander':['santander'],'BBVA':['bbva'],'Bankinter':['bankinter'],'Bankia':['bankia'],
                 'Sabadell':['sabadell'],'ING':['ing'],'Abanca':['abanca'],'Deutsche Bank':['deutsche'],
                  'CaixaBank':['caixabank'],'Openbank':['openbank']}
    for key,values in dict_banks.items():
        for i in values:
            if string_found(i,text.lower()):
                tags.append(key)
                break
    return tags

In [20]:
def economista_news(economista_entries,word_count=100):
    economista = {}
    for i in range(len(economista_entries)):
        print(i)
        noticia = {'newspaper':None,'title':None,'headline':None,'summarise':None,'date':None,'link':None,
                   'text':None,'current_date':None,'img':None,'premium':None,'tag':None}
        # Periodico
        noticia['newspaper'] = find_between(s=economista_entries[i]['id'], first='www.el', last='.' )
        # Title
        noticia['title'] = economista_entries[i]['title']
        # Headline
        headline =  re.sub(pattern='&quot;',repl='"',string=economista_entries[i]['summary_detail']['value'])
        noticia['headline'] = headline
        # Date
        noticia['date'] = {'date1':economista_entries[i]['published'],'date2':economista_entries[i]['published_parsed']}
        # Link
        noticia['link'] = economista_entries[i]['id']
        # Text
        text = get_only_text(economista_entries[i]['id'])[0][len(headline)+1:]
        if  text.find('\n\t\t\t\tif') != -1:
            index =  text.find('\n\t\t\t\tif')
            text = text[:index]
        noticia['text'] = text
        # Bank Tags
        noticia['tag'] = bank_tags(text)
        # Summarise
        noticia['summarise'] = summarize(text, word_count=word_count)
        # Current Date
        noticia['current_date'] = datetime.datetime.now().timetuple()
        # Image
        noticia['img'] = get_img(economista_entries[i]['id'])
    return economista

## 2. RSS Economista

In [21]:
url_economista = 'https://www.eleconomista.es/rss/rss-empresas.php'
economista = feedparser.parse(url_economista)
economista_entries = economista['entries']

print('Total Entradas Economista:',len(economista_entries))

Total Entradas Economista: 12


In [22]:
economista = economista_news(economista_entries,word_count=100)

0
1
2
3
4
5
6
7
8
9
10
11


In [25]:
yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
before_yes = datetime.datetime.today() - datetime.timedelta(days=2)

for i in list(economista.keys()):
    if economista[i]['date']['date2'][2] in [datetime.datetime.today().day,yesterday.day,before_yes.day]:
        print(colored((economista[i]['newspaper']).upper(),'green',attrs=['bold']))
        print(colored(economista[i]['date']['date1'],'grey',attrs=['bold']))
        print(colored(economista[i]['title'],'magenta',attrs=['bold']))
        print(colored('Headline','blue'))
        print(economista[i]['headline'])
        print(colored('Summarise','blue'))
        
        if economista[i]['summarise'] != 'Premium Content':
            print(economista[i]['summarise'])
        else:
            print(colored(economista[i]['summarise'],'red'))
        print(colored('Text','blue'))
        print(economista[i]['text'])
        url = cincod[i]['img']
        display(Image(url=url,width=500))
        display(Markdown('<a href="'+economista[i]['link']+'">Link</a>'))
        print('\n')